<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

<img src="mioti.png" style="height: 100px">
<center style="color:#888">Data Science with Python @ Data Science for IoT</center>

# DSPy7 Challenge. Web scraping

Vamos a intentar extraer datos de una web de una librería, que no es más que una web preparada para testear scrapping contra ella. 

<img src="books_to_scrape.png" style="height: 500px">

Empezaremos concretamente en esta url: http://books.toscrape.com/catalogue/page-1.html

In [1]:
# instalar requisitos
import sys
# !{sys.executable} -m pip install folium ortools==6.7.4973

!{sys.executable} -m pip install scrapy

**Inicialización**

In [2]:
import logging
import re
import pandas as pd
import json
import scrapy
from scrapy.crawler import CrawlerProcess
# Para ir viendo la salida de los comandos a medida que ocurre
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Esqueleto de los web scrapers**

In [3]:
class JsonWriterPipeline(object):
    @classmethod
    def from_crawler(cls, crawler):
        settings = crawler.settings
        file_name = settings.get("FILE_NAME")
        return cls(file_name)
    
    def __init__(self, file_name):
        self.file_name = file_name

    def open_spider(self, spider):
        print(self.file_name)
        self.file = open(self.file_name, 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [4]:
class SimpleBookSpider(scrapy.Spider):
    name = "simplebooks"
    start_urls = [
        'http://books.toscrape.com/catalogue/page-1.html',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},     
        'FILE_NAME': "simplebook.jl"
    }
    
    def parse(self, response):
        """En este método indicamos cuáles son los elementos que nos interesan"""
        results = response.css('article.product_pod')
        for book in results:
            yield{
                #'book': quote.css('span.text::text').extract_first(),
                'title': book.css('a::attr(title)').extract_first(),
                'price': book.css('p.price_color::text').extract_first(),
            }
        pass

In [5]:
#creamos una nueva clase para sacar titulo,precio y rating
class find_BookSpider(scrapy.Spider):
    name = "ratingbooks" 
    start_urls = [
        'http://books.toscrape.com/catalogue/page-1.html',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},     
        'FILE_NAME': "ratingbook.jl"
    }
    
    def parse(self, response):
        results = response.css('article.product_pod')
        for article in results:
            yield{
                'title': article.css('a::attr(title)').extract_first(),
                'price': article.css('p.price_color::text').extract_first(),
                'rating' : article.css('p::attr(class)').extract_first(),
            }

In [6]:
process = CrawlerProcess()
process.crawl(SimpleBookSpider)
process.crawl(find_BookSpider)
#process.crawl(RatingBookSpider)
#process.crawl(BookSpider)
#process.crawl(FinalBookSpider)
process.start()

2019-06-08 19:04:46 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2019-06-08 19:04:46 [scrapy.utils.log] INFO: Versions: lxml 4.3.3.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 19.2.0, Python 3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.6.1, Platform Darwin-18.6.0-x86_64-i386-64bit
2019-06-08 19:04:46 [scrapy.crawler] INFO: Overridden settings: {'LOG_LEVEL': 30}


simplebook.jl


ratingbook.jl


**Ejercicio 1 (0.7p):** Obten en un dataframe una lista de los libros en esta página (no en toda la librería), con título y precio.

In [7]:
dfjason = pd.read_json('simplebook.jl', lines=True)
dfjason

,price,title
0,£51.77,A Light in the Attic
1,£53.74,Tipping the Velvet
2,£50.10,Soumission
3,£47.82,Sharp Objects
4,£54.23,Sapiens: A Brief History of Humankind
5,£22.65,The Requiem Red
6,£33.34,The Dirty Little Secrets of Getting Your Dream...
7,£17.93,The Coming Woman: A Novel Based on the Life of...
8,£22.60,The Boys in the Boat: Nine Americans and Their...
9,£52.15,The Black Maria


In [8]:
dfjason.title[0]

'A Light in the Attic'

**Ejercicio 2 (0.3p):** Ahora captura también el rating de los libros. ¡Ordénalos por rating!

In [9]:
dfjason = pd.read_json('ratingbook.jl', lines=True)
dfjason

,price,rating,title
0,£51.77,star-rating Three,A Light in the Attic
1,£53.74,star-rating One,Tipping the Velvet
2,£50.10,star-rating One,Soumission
3,£47.82,star-rating Four,Sharp Objects
4,£54.23,star-rating Five,Sapiens: A Brief History of Humankind
5,£22.65,star-rating One,The Requiem Red
6,£33.34,star-rating Four,The Dirty Little Secrets of Getting Your Dream...
7,£17.93,star-rating Three,The Coming Woman: A Novel Based on the Life of...
8,£22.60,star-rating Four,The Boys in the Boat: Nine Americans and Their...
9,£52.15,star-rating One,The Black Maria


In [10]:
dfjason['rating'] = dfjason['rating'].str.replace('star-rating ', '')
dfjason['rating'] = dfjason['rating'].replace('Zero', 0)
dfjason['rating'] = dfjason['rating'].replace('One', 1)
dfjason['rating'] = dfjason['rating'].replace('Two', 2)
dfjason['rating'] = dfjason['rating'].replace('Three', 3)
dfjason['rating'] = dfjason['rating'].replace('Four', 4)
dfjason['rating'] = dfjason['rating'].replace('Five', 5)

In [11]:
dfjason.sort_values(by='rating', ascending = False)

,price,rating,title
12,£17.46,5,Set Me Free
4,£54.23,5,Sapiens: A Brief History of Humankind
14,£35.02,5,Rip it Up and Start Again
13,£52.29,5,Scott Pilgrim's Precious Little Life (Scott Pi...
3,£47.82,4,Sharp Objects
6,£33.34,4,The Dirty Little Secrets of Getting Your Dream...
8,£22.60,4,The Boys in the Boat: Nine Americans and Their...
11,£20.66,4,Shakespeare's Sonnets
0,£51.77,3,A Light in the Attic
15,£57.25,3,Our Band Could Be Your Life: Scenes from the A...


**Ejercicio 3 (opcional, 0.3p):** Vayamos un poco más lejos y expandamos el mismo Spider para que siga los enlaces de siguiente. Obten una lista de las 3 primeras páginas con libros y construye un dataframe en el que además, por cada libro, aparezca el número de página en que aparece. ¡¡¡Prohibido usar "start_urls"!!!

In [12]:
#Scrapear lo mismo que antes, pero dandole a next (POST); 
#def parse(request)
    #yield()
    
    
    #yield request.follow(url_to_scrape, method_for_parsing, self.parse())

**Ejercicio 4 (opcional, 0.7p):** El spider definitivo. Construye una clase que vaya categoria por categoría de libros (para hacer la ejecución más agil, basta con que solo lo haga con las 4 primeras). Por cada categoría, que entre en cada libro y obtenga:
* Título
* Precio
* Libros en stock
* Rating
* UPC
* Imagen

Ojo, porque una categoría puede tener más de una página y quiero los libros de todas 😠.

In [13]:
# rqquest_follow (Self.parse(nueva funcion))

#nueva funcion = parse.book (estructura en la nueva pagina)

#request.meta (propiedad para leer el response)